Vamos a ilustrar como paralelizar bucles for en Python con un ejemplo:

La idea es paralelizar el siguiente algoritmo KNN para clasificacion basado en la distancia de Gower-BQ (una variante de la distancia de Gower aplicable a conjunto de datos binarios-cuantitativos).

Aqui lo importante no es el funcionamiento del algoritmo en si, si no que hay una parte de este que genera unos costes computacionales altos, lo que se traduce en que al aplicar el algoritmo a un data-set de 5000 variables, este tarde mas de 13 minutos en compilarse por completo. La idea de paralelizarlo es reducir este tiempo de cómputo, para asi hacerlo mas aplicable en contextos reales.

El algoritmo original (sin paralelizar):

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def KNN_classification( X , Y , x_new, k, distance = "Gower-BQ" ,  p1=0, p2=0 ):

 
####################################################################################################################################################################################################################################################

    X = pd.concat([X, x_new.to_frame().T], ignore_index=True)

    distances = []

    groups_knn = []

######################################################################################################################################


    if distance == "Gower-BQ":

        def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

##########################################################################################

        def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)

##########################################################################################

        def GowerBQ_Similarity_Python(i,j, BQ_Data_Set, p1, p2):

            X = BQ_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical 

##########################################################################################
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0, p1)

            for r in range(0, p1): 

                G_vector[r] = G(r, X)
##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
         
 
##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] 
     
            numerator = numerator_part_1 + numerator_part_2

            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1])  

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################


        def Dist_GowerBQ_Py(i, j, BQ_Data ,  p1, p2):

            Dist_Gower = np.sqrt( 1 - GowerBQ_Similarity_Python(i, j, BQ_Data , p1, p2) )

            return(Dist_Gower)

##########################################################################################

        for i in range(1, len(X)):

            distances.append( Dist_GowerBQ_Py( len(X), i , X, p2, p3) )


######################################################################################################################################

######################################################################################################################################
    
    distances = pd.DataFrame({'distances': distances})

    distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
    knn = distances.iloc[0:k , :]

    for i in knn.iloc[:,0]:

        groups_knn.append(Y[i])

    unique, counts = np.unique(groups_knn , return_counts=True)

    unique_Y , counts_Y = np.unique(Y , return_counts=True)

    if len(unique) == len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique_Y })
    
    elif len(unique) < len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique })



    prediction_group = proportions_groups_knn.sort_values(by=["proportions_groups"], ascending=False).iloc[0,:]['groups']

    message = print( "x_new is classify in the group", prediction_group , ". So KNN algorithm predict y_new =",  prediction_group )                                      
                                       

    return proportions_groups_knn , message  

Vamos a probar el algoritmo con un mismo data-set pero con distintos tamaños de filas (observaciones).

In [104]:
Gender_classification = pd.read_csv('gender_classification.csv')

x_new = pd.Series({'long_hair': 1, 'forehead_width_cm': 4, 'forehead_height_cm': 6, 'nose_wide': 1 , 'nose_long': 1 , 'nose_long': 1 , 'lips_thin':1, 'distance_nose_to_lip_long': 1 })

Y = Gender_classification.loc[: , ['gender']]


In [105]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

Y['gender'] = ord_enc.fit_transform(Y[['gender']])

In [106]:
X = Gender_classification.loc[: , ['forehead_width_cm', 'forehead_height_cm',   # Quantitative (2)

                   'long_hair', 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long'     # Binary (5)
                 
                            ]]  

X

,forehead_width_cm,forehead_height_cm,long_hair,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
0,11.8,6.1,1,1,0,1,1
1,14.0,5.4,0,0,0,1,0
2,11.8,6.3,0,1,1,1,1
3,14.4,6.1,0,0,1,1,1
4,13.5,5.9,1,0,0,0,0
...,...,...,...,...,...,...,...
4996,13.6,5.1,1,0,0,0,0
4997,11.9,5.4,1,0,0,0,0
4998,12.9,5.7,1,0,0,0,0
4999,13.2,6.2,1,0,0,0,0


In [107]:
X = pd.concat([X, x_new.to_frame().T], ignore_index=True)
X

,forehead_width_cm,forehead_height_cm,long_hair,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long
0,11.8,6.1,1,1,0,1,1
1,14.0,5.4,0,0,0,1,0
2,11.8,6.3,0,1,1,1,1
3,14.4,6.1,0,0,1,1,1
4,13.5,5.9,1,0,0,0,0
...,...,...,...,...,...,...,...
4997,11.9,5.4,1,0,0,0,0
4998,12.9,5.7,1,0,0,0,0
4999,13.2,6.2,1,0,0,0,0
5000,15.4,5.4,1,1,1,1,1


In [108]:

distances = []

groups_knn = []

In [109]:
def a(Binary_Data) :

            X = Binary_Data

            a = X @ X.T

            return(a)

In [110]:
def d(Binary_Data):

            X = Binary_Data

            ones_matrix = np.ones(( X.shape[0] , X.shape[1])) 

            d = (ones_matrix - X) @ (ones_matrix - X).T

            return(d)


In [111]:
def GowerBQ_Similarity_Python(i,j, BQ_Data_Set, p1, p2):

            X = BQ_Data_Set

   # The data matrix X have to be order in the following way:
   # The p1 first are quantitative, the following p2 are binary categorical 

##########################################################################################
            def G(k, X):

                range = X.iloc[:,k].max() - X.iloc[:,k].min() 

                return(range)

            G_vector = np.repeat(0, p1)

            for r in range(0, p1): 

                G_vector[r] = G(r, X)
##########################################################################################
    
            ones = np.repeat(1, p1)

            Quantitative_Data = X.iloc[: , 0:p1]

            Binary_Data = X.iloc[: , (p1):(p1+p2)]
         
 
##########################################################################################

            numerator_part_1 = ( ones - ( (Quantitative_Data.iloc[i-1,:] - Quantitative_Data.iloc[j-1,:]).abs() / G_vector ) ).sum() 

            numerator_part_2 = a(Binary_Data).iloc[i-1,j-1] 
     
            numerator = numerator_part_1 + numerator_part_2

            denominator = p1 + (p2 - d(Binary_Data).iloc[i-1,j-1])  

            Similarity_Gower = numerator / denominator  

            return(Similarity_Gower)

##########################################################################################


def Dist_GowerBQ_Py(i, j, BQ_Data ,  p1, p2):

            Dist_Gower = np.sqrt( 1 - GowerBQ_Similarity_Python(i, j, BQ_Data , p1, p2) )

            return(Dist_Gower)

In [112]:
p2=2

In [113]:
p3=5

In [114]:
distances = ( Parallel( n_jobs=n_jobs )( delayed( Dist_GowerBQ_Py )( len(X), i , X, p2, p3 ) for i in range(1, 100) ) )

In [115]:
k=5

In [116]:
distances

[0.5012970190403103,
 0.8626446801295676,
 0.5153488290317692,
 0.6541575329552339,
 0.8378234014087049,
 0.4171642051110977,
 0.5512677243373713,
 0.9425028850415729,
 0.5369466345100676,
 0.848221975358172,
 0.544154294390751,
 0.4045199174779453,
 0.5822036061044615,
 0.5737776391780872,
 0.8600060404499433,
 0.5571262068774098,
 0.38138503569823684,
 0.8416897043694657,
 0.8550841076577386,
 0.553618556854362,
 0.3988620176087328,
 0.4309458036856673,
 0.6556448199064261,
 0.6262649536799837,
 0.946970562770167,
 0.8370479978169416,
 0.8474560862851941,
 0.8596284307542643,
 0.560032466591325,
 0.41009976613236226,
 0.45154856205739813,
 0.4061219828266686,
 0.7614920418954685,
 0.7279217882919358,
 0.5465357250000211,
 0.3719027941703158,
 0.8288624656799143,
 0.7891307069980693,
 0.9459414330735253,
 0.560032466591325,
 0.8424608357308129,
 0.4384150912181198,
 0.3370999312316209,
 0.8265088524128965,
 0.5290275322420657,
 0.40291148201269017,
 0.8734912781402073,
 0.679476498260

In [119]:
distances = pd.DataFrame({'distances': distances}, index=range(0,99))


In [120]:
distances

,distances
0,0.501297
1,0.862645
2,0.515349
3,0.654158
4,0.837823
...,...
94,0.533915
95,0.562924
96,0.347533
97,0.448663


In [121]:

distances = distances.sort_values(by=["distances"]).reset_index(drop=False)
        
knn = distances.iloc[0:k , :]

for i in knn.iloc[:,0]:

        groups_knn.append(Y.iloc[i,:])

unique, counts = np.unique(groups_knn , return_counts=True)

unique_Y , counts_Y = np.unique(Y , return_counts=True)

if len(unique) == len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique_Y })
    
elif len(unique) < len(unique_Y) :

        proportions_groups_knn = pd.DataFrame({'proportions_groups': counts/k, 'groups': unique })



prediction_group = proportions_groups_knn.sort_values(by=["proportions_groups"], ascending=False).iloc[0,:]['groups']


In [ ]:
KNN_classification( X.iloc[0:100,:] , Y.iloc[0:100,:]  , x_new, 5 , distance = "Gower-BQ" , p1=2, p2=5 )

NameError: name 'KNN_classification' is not defined

In [ ]:
from joblib import Parallel, delayed
import multiprocessing


In [ ]:
def suma(x , y):

    sum =((x+y)^(1000*(x+y)))   

    return(sum)

In [ ]:
 vector = []

for x in range(0,10):

      vector.append( suma(x,15) )

In [ ]:
vector

[14999, 16016, 17017, 17986, 18987, 20020, 21021, 21990, 22991, 24024]

In [ ]:

n_jobs  = multiprocessing.cpu_count()


In [ ]:
n_jobs

8

In [ ]:
Parallel(n_jobs=n_jobs)(delayed(suma)(x, 15) for x in range(0,10))

[14999, 16016, 17017, 17986, 18987, 20020, 21021, 21990, 22991, 24024]

https://www.cienciadedatos.net/documentos/py12-paralelizar-con-python.html